### Install and Import

In [3]:
pip install tensorflow==2.11.0 opencv-python mediapipe scikit-learn matplotlib


  Using cached mediapipe-0.10.11-cp39-cp39-win_amd64.whl (50.8 MB)
  Using cached sounddevice-0.4.6-py3-none-win_amd64.whl (199 kB)
  Using cached opencv_contrib_python-4.9.0.80-cp37-abi3-win_amd64.whl (45.3 MB)
  Using cached jax-0.4.25-py3-none-any.whl (1.8 MB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\avani\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [1]:
import cv2
import numpy as np
import os
import time
from matplotlib import pyplot as plt
import mediapipe as mp
from datetime import datetime

### Drawing Keypoints on Video Feed

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [7]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
  while cap.isOpened(): #always

    ret, frame = cap.read()

    image, results = mediapipe_detection(frame, holistic)

    draw_styled_landmarks(image, results)

    cv2.imshow('OpenCV Feed', image)
    if cv2.waitKey(10) & 0xFF == ord('q'):
      break
  cap.release()
  cv2.destroyAllWindows()

In [6]:
len(results.face_landmarks.landmark)

NameError: name 'results' is not defined

In [ ]:
draw_landmarks(frame, results)

In [7]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

NameError: name 'frame' is not defined

### Extracting Keypoints

In [8]:
len(results.pose_landmarks.landmark)

NameError: name 'results' is not defined

In [9]:
len(results.face_landmarks.landmark)

NameError: name 'results' is not defined

In [10]:
len(results.left_hand_landmarks.landmark)

NameError: name 'results' is not defined

In [11]:
len(results.right_hand_landmarks.landmark)

NameError: name 'results' is not defined

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [7]:
extract_keypoints(results)[:5]

NameError: name 'results' is not defined

### Folders for Image's NumPy Array Storage

In [8]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['hello', 'thanks', 'iloveyou', 'yes', 'no'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [101]:
for action in actions: 
    dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    # Folder start
    start_folder = dirmax
    print(dirmax)
    temp_strg=DATA_PATH+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    for sequence in range(1,no_sequences+1):
        try: 
            os.makedirs(os.path.join(temp_strg, action, str(dirmax+sequence)))
        except:
            pass

89
89
89
89
89


In [17]:
# for action in actions: 
#     for sequence in range(no_sequences):
#         try: 
#             os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
#         except:
#             pass

### Collecting Keypoints for Training

In [15]:
def make_npy_array(action):

    cap = cv2.VideoCapture(0)
    # Set mediapipe model 
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        
        # NEW LOOP
        # Loop through actions
        # for action in actions:
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(temp_strg, action, str(sequence+1), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                        
        cap.release()
        cv2.destroyAllWindows()

In [104]:
# for hello
make_npy_array(actions[0])

In [105]:
# for thanks
make_npy_array(actions[1])

In [106]:
# for iloveyou
make_npy_array(actions[2])

In [107]:
# for yes
make_npy_array(actions[3])

In [108]:
# for no
make_npy_array(actions[4])

### Preprocessing and EDA

In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2, 'yes': 3, 'no': 4}

In [11]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [12]:
np.array(sequences).shape

# (no_of_videos, no_of_frames_in_each_video, no_of_keypoints_in_each_frame)

(600, 30, 1662)

In [13]:
np.array(labels).shape

# (no_of_labels) same as number of videos

(600,)

In [14]:
X = np.array(sequences)

In [15]:
y = to_categorical(labels).astype(int)
y

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1]])

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, shuffle=True)

In [17]:
print("Training set shape:", X_train.shape, y_train.shape)
print("Test set shape:", X_test.shape, y_test.shape)

Training set shape: (450, 30, 1662) (450, 5)
Test set shape: (150, 30, 1662) (150, 5)


### Building the LSTM NN

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard
from keras import regularizers

In [19]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [20]:
# overfitting

# modelLSTM = Sequential()
# modelLSTM.add(LSTM(128, return_sequences=True, activation='tanh', input_shape=(30, 1662)))
# modelLSTM.add(Dropout(0.2))  # Add dropout after the first LSTM layer
# modelLSTM.add(LSTM(128, return_sequences=True, activation='tanh'))
# modelLSTM.add(Dropout(0.2))  # Add dropout after the second LSTM layer
# modelLSTM.add(LSTM(128, return_sequences=False, activation='tanh'))
# modelLSTM.add(Dropout(0.2))  # Add dropout after the third LSTM layer
# modelLSTM.add(Dense(64, activation='relu'))
# modelLSTM.add(Dense(32, activation='relu'))
# modelLSTM.add(Dense(5, activation='softmax'))


In [21]:
modelLSTM = Sequential()
modelLSTM.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
modelLSTM.add(LSTM(128, return_sequences=True, activation='relu'))
modelLSTM.add(LSTM(64, return_sequences=False, activation='relu'))
modelLSTM.add(Dense(64, activation='relu'))
modelLSTM.add(Dense(32, activation='relu'))
modelLSTM.add(Dense(actions.shape[0], activation='softmax'))

In [22]:
modelLSTM.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [23]:
modelLSTM.fit(X_train, y_train, epochs=450, batch_size=32, callbacks=[tb_callback])

Epoch 1/450
15/15 [==============================] - 5s 67ms/step - loss: 2.2823 - categorical_accuracy: 0.2156
Epoch 2/450
15/15 [==============================] - 1s 67ms/step - loss: 1.5539 - categorical_accuracy: 0.2489
Epoch 3/450
15/15 [==============================] - 1s 78ms/step - loss: 1.5577 - categorical_accuracy: 0.3089
Epoch 4/450
15/15 [==============================] - 1s 69ms/step - loss: 1.3501 - categorical_accuracy: 0.3733
Epoch 5/450
15/15 [==============================] - 1s 67ms/step - loss: 1.2729 - categorical_accuracy: 0.4333
Epoch 6/450
15/15 [==============================] - 1s 70ms/step - loss: 1.4769 - categorical_accuracy: 0.3422
Epoch 7/450
15/15 [==============================] - 1s 65ms/step - loss: 1.2421 - categorical_accuracy: 0.4311
Epoch 8/450
15/15 [==============================] - 1s 65ms/step - loss: 1.3585 - categorical_accuracy: 0.4067
Epoch 9/450
15/15 [==============================] - 1s 71ms/step - loss: 1.2534 - categorical_accuracy:

In [24]:
modelLSTM.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 5)                 165       
                                                                 
Total params: 596,741
Trainable params: 596,741
Non-trai

### Predictions

In [25]:
res = modelLSTM.predict(X_test)
res

5/5 [==============================] - 1s 22ms/step


array([[9.63154972e-01, 3.36743426e-07, 1.34127471e-03, 2.89338734e-02,
        6.56944280e-03],
       [2.83568170e-06, 5.99111943e-03, 2.66806805e-03, 8.27920973e-01,
        1.63416952e-01],
       [2.55643857e-12, 2.39359314e-13, 1.91372055e-06, 1.04890845e-03,
        9.98949230e-01],
       [9.42256093e-01, 3.41513987e-06, 3.71782742e-02, 1.38385100e-02,
        6.72376156e-03],
       [4.46570496e-08, 9.99822438e-01, 9.41788094e-05, 7.22760233e-05,
        1.11380659e-05],
       [2.49357778e-03, 6.26674882e-05, 9.94430721e-01, 1.50665268e-03,
        1.50638469e-03],
       [6.89911425e-01, 2.16966673e-05, 8.72817822e-03, 2.05239430e-01,
        9.60992798e-02],
       [4.89486456e-06, 5.28325108e-06, 2.83578760e-04, 4.10296470e-01,
        5.89409828e-01],
       [9.44690704e-01, 3.48575645e-06, 2.33749207e-02, 2.22729761e-02,
        9.65795573e-03],
       [3.26494424e-04, 1.67901177e-04, 9.93022501e-01, 2.87712947e-03,
        3.60601256e-03],
       [8.61898899e-01, 8.1885

### Saving Weights

In [26]:
modelLSTM.save('modelWeight'+str(X.shape[0])+'.h5')

### Evaluation

In [27]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [28]:
yhat = modelLSTM.predict(X_test)

5/5 [==============================] - 0s 21ms/step


In [29]:
ytrue = np.argmax(y_test, axis=1).tolist()
ytrue

[0,
 3,
 4,
 0,
 1,
 2,
 0,
 3,
 0,
 2,
 0,
 0,
 0,
 4,
 3,
 1,
 0,
 1,
 1,
 0,
 0,
 4,
 4,
 4,
 3,
 4,
 0,
 1,
 4,
 3,
 4,
 1,
 0,
 2,
 1,
 3,
 1,
 4,
 3,
 2,
 3,
 0,
 4,
 4,
 0,
 1,
 3,
 0,
 0,
 4,
 2,
 2,
 0,
 4,
 4,
 2,
 1,
 1,
 1,
 1,
 0,
 1,
 2,
 4,
 4,
 0,
 4,
 4,
 0,
 2,
 2,
 0,
 3,
 3,
 3,
 4,
 2,
 1,
 4,
 3,
 2,
 3,
 4,
 3,
 0,
 0,
 1,
 1,
 3,
 1,
 1,
 4,
 0,
 2,
 2,
 2,
 1,
 0,
 3,
 0,
 0,
 1,
 4,
 2,
 0,
 3,
 1,
 2,
 2,
 3,
 1,
 3,
 4,
 4,
 3,
 3,
 2,
 1,
 4,
 3,
 1,
 0,
 3,
 4,
 1,
 1,
 4,
 2,
 4,
 3,
 0,
 4,
 1,
 1,
 3,
 1,
 0,
 0,
 2,
 0,
 0,
 1,
 1,
 0,
 4,
 0,
 3,
 3,
 0,
 0]

In [30]:
yhat = np.argmax(yhat, axis=1).tolist()
yhat

[0,
 3,
 4,
 0,
 1,
 2,
 0,
 4,
 0,
 2,
 0,
 4,
 0,
 4,
 2,
 2,
 0,
 1,
 1,
 0,
 0,
 4,
 3,
 4,
 3,
 3,
 0,
 1,
 4,
 4,
 4,
 1,
 0,
 2,
 1,
 3,
 1,
 3,
 4,
 2,
 3,
 0,
 4,
 4,
 0,
 1,
 4,
 0,
 0,
 4,
 2,
 2,
 0,
 4,
 3,
 2,
 1,
 1,
 1,
 1,
 0,
 1,
 2,
 3,
 4,
 0,
 4,
 4,
 0,
 2,
 2,
 0,
 3,
 3,
 4,
 3,
 2,
 1,
 4,
 3,
 2,
 3,
 4,
 3,
 0,
 0,
 1,
 1,
 3,
 1,
 1,
 3,
 2,
 2,
 2,
 2,
 1,
 0,
 3,
 0,
 0,
 1,
 3,
 2,
 0,
 3,
 1,
 2,
 2,
 3,
 1,
 3,
 4,
 4,
 3,
 3,
 2,
 1,
 4,
 3,
 1,
 0,
 3,
 4,
 1,
 1,
 4,
 2,
 4,
 3,
 0,
 3,
 1,
 1,
 3,
 1,
 0,
 0,
 2,
 0,
 0,
 1,
 1,
 0,
 3,
 0,
 4,
 4,
 0,
 2]

In [31]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[112,   0],
        [  3,  35]],

       [[118,   0],
        [  1,  31]],

       [[125,   4],
        [  0,  21]],

       [[112,  10],
        [  8,  20]],

       [[111,   8],
        [ 10,  21]]], dtype=int64)

In [32]:
accuracy_score(ytrue, yhat)

0.8533333333333334

### Real-Time

In [33]:
from scipy import stats

In [34]:
# colors = [(245,117,16), (117,245,16), (16,117,245)]
# def prob_viz(res, actions, input_frame, colors):
#     output_frame = input_frame.copy()
#     for num, prob in enumerate(res):
#         cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
#         cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
#     return output_frame

In [35]:
def prob_viz(res, actions, input_frame):
    output_frame = input_frame.copy()

    # Get indices of top 2 probabilities
    top_idx = res.argsort()[-2:][::-1]

    # Display message asking user to choose between the top 2 actions
    cv2.rectangle(output_frame, (0, output_frame.shape[0] - 45), (output_frame.shape[1], output_frame.shape[0]), (0, 0, 0), -1)
    cv2.putText(output_frame, f"Did you mean {actions[top_idx[0]]} ({res[top_idx[0]]*100:.0f}%) or {actions[top_idx[1]]} ({res[top_idx[1]]*100:.0f}%)?", (10, output_frame.shape[0] - 18), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

    # Open a new window and show the image corresponding to the action with the second highest probability
    img_path = os.path.join("images", f"{actions[top_idx[1]]}.png")
    img = cv2.imread(img_path)
    img = cv2.resize(img, (500, 310))  # Resize the image

    cv2.rectangle(img, (0, img.shape[0] - 45), (img.shape[1], img.shape[0]), (0, 0, 0), -1)
    cv2.putText(img, f" Correct Way to Act: {actions[top_idx[1]]}", (0, img.shape[0] - 18), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)  # Add text to the image
    cv2.imshow("Suggested Action", img)

    return output_frame

In [36]:
# 1. New detection variables
from urllib.request import AbstractBasicAuthHandler


sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        # print(results)
        
        # Draw landmarks
        # draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = modelLSTM.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('SignLearn', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 24ms/step
yes
1/1 [==============================] - 0s 20ms/step
yes
1/1 [==============================] - 0s 20ms/step
yes
1/1 [==============================] - 0s 19ms/step
yes
1/1 [==============================] - 0s 27ms/step
yes
1/1 [==============================] - 0s 22ms/step
yes
1/1 [==============================] - 0s 22ms/step
yes
1/1 [==============================] - 0s 21ms/step
yes
1/1 [==============================] - 0s 22ms/step
yes
1/1 [==============================] - 0s 21ms/step
yes
1/1 [==============================] - 0s 20ms/step
yes
1/1 [==============================] - 0s 22ms/step
yes
1/1 [==============================] - 0s 22ms/step
yes
1/1 [==============================] - 0s 21ms/step
yes
1/1 [==============================] - 0s 21ms/step
yes
1/1 [==============================] - 0s 27ms/step
yes
1/1 [==============================] - 0s 20ms/step
yes
1/1 [==============================] - 0s 20ms/s